In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.plots import plots_dev
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
from plaster.run.sigproc_v2.sigproc_v2_params import SigprocV2Params
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/val_calib")
run = job.runs[0]

In [ ]:
z.im(run.ims_import.ims[0, 0, 30], _cspan=(150, 400))

In [ ]:
# TODO: Multichannel
z_reg_psf = np.array(run.sigproc_v2.calib["regional_psf_zstack.instrument_channel[0]"])

In [ ]:
# run.sigproc_v2.calib.save("val.calib")

In [ ]:
z.im(np.array(run.sigproc_v2.calib["regional_illumination_balance.instrument_channel[0]"]))

In [ ]:
psf_im = z_reg_psf[6, :, :, :, :]
plots_dev.plot_psfs(psf_im, _size=300)

In [ ]:
z.im(psf_im[2,2])
params, fit_cov = imops.fit_gauss2(psf_im[2,2])
fit_im = imops.gauss2_rho_form(*params)
# np.set_printoptions(precision=4)
perfect_psf = fit_im - params[6]
z.im(perfect_psf)

In [ ]:
a = np.array(run.sigproc_v2.calib["regional_psf_zstack.instrument_channel[0]"])
a[:] = perfect_psf[None, None, None, :, :]
run.sigproc_v2.calib.save("val.calib")

In [ ]:

run.sigproc_v2.calib["regional_psf_zstack.instrument_channel[0]"] = 
a.shape

In [ ]:
cen = z_reg_psf.shape[-1] // 2
psf_im = np.moveaxis(z_reg_psf[1:12, :, :, cen, :], (0), (2))
plots_dev.plot_psfs(psf_im, _size=600)

In [ ]:
for zi in range(13):
    plots_dev.plot_psfs(z_reg_psf[zi], _size=300, f_title=f"zi={zi}")

In [ ]:
z.im(z_reg_psf[12, 2, 2])